In [ ]:
%load_ext autoreload
%autoreload 2

import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.path.abspath(''), '..')))

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from hydracv.midline import find_midline_midpoints, find_midline_midpoints_corrected
from hydracv.fluorescence import trace_fluo
import hydracv.utils.utils as utils

In [ ]:
BISECTION_DATA_HOME = "/Users/katyabrooun/Google Drive/Shared drives/hydra/data-analysis/Bisection_Full/"
VIDEO_NAME = "71420_ngcamp_hy6_peduncle48hr" # TO MODIFY
FPS = 5 # frames per second in original video (used to convert frames to seconds for x axes on plots)
BODY_PART = '/Peduncle/' # TO MODIFY
EXP_NAME = 'GCaMP6_Bisection_Exp6'# TO MODIFY
SAVE_DIR = '/Users/katyabrooun/Fairhall_Lab/hydra/bisection-data-analysis/exp6' + BODY_PART # TO MODIFY

In [ ]:
# for saving off data
FILE_FLUO = SAVE_DIR + 'total_fluo_' + VIDEO_NAME + '.csv'
FILE_MIDPOINTS = SAVE_DIR + "midpoints_"+ VIDEO_NAME +".csv"
FILE_LENGTH = SAVE_DIR + "lengths_" + VIDEO_NAME + ".csv"

## Midline and length

Requirements: 
- per-frame contours
- tracked data (results from DeepLabCut)

In [ ]:
FILE_CONTOUR = BISECTION_DATA_HOME + 'Icy_Contours/' + EXP_NAME + BODY_PART + VIDEO_NAME +  "_ROIs.xml"
FILE_MARKER = BISECTION_DATA_HOME + 'DLC_Data/' + EXP_NAME + BODY_PART + \
"71420_ngcamp_hy6_peduncle48hrDLC_resnet50_EXP6-Ped-48hrNov30shuffle1_45000.csv" # TO MODIFY

In [ ]:
# Original find midline code
midpoints_orig = find_midline_midpoints.find_midline(file_contour=FILE_CONTOUR,
                                                     file_marker=FILE_MARKER,
                                                     nseg=40,
                                                     play=False)

In [ ]:
# Save the original midpoints
df = pd.DataFrame(midpoints_orig)
df.to_csv(FILE_MIDPOINTS, index=False)

In [ ]:
# Plot the length
midpoints_orig_data = find_midline_midpoints_corrected.load_midpoints(FILE_MIDPOINTS)

midlens_orig = find_midline_midpoints_corrected.extract_lengths(midpoints_orig_data, normalize=False)

plt.figure(figsize=(20,3))
plt.plot(midlens_orig)
plt.show()

In [ ]:
# Save the lengths (per frame)
df = pd.DataFrame(midlens_orig)
df.to_csv(FILE_LENGTH, index=False)

### First derivative of midline length

In [ ]:
# Plot the change in length, rather than length itself

midlen_grads = np.gradient(midlens_orig)

plt.figure(figsize=(20,3))
plt.plot(midlen_grads)
plt.show()

## Fluorescence

Requirements: 
- video

Optional:
- single ROI specifying which portion of video frames to compute fluorescence over (useful if there are external objects in video)

In [ ]:
FILE_VIDEO = BISECTION_DATA_HOME + "Original_Videos/" + EXP_NAME + BODY_PART + VIDEO_NAME + ".avi"
FILE_FLUO_ROI = BISECTION_DATA_HOME + 'Icy_Contours/' + EXP_NAME + BODY_PART + VIDEO_NAME +  "_max_intensity_ROI.xml"

In [ ]:
# Optional (use if slide appears in video partway through)
ROI = []
ROI = find_midline_midpoints.load_contour(FILE_FLUO_ROI)

In [ ]:
# Calculate fluorescence intensity for each video frame
fluo = trace_fluo.trace(FILE_VIDEO, display=False, ROI=ROI)

In [ ]:
# Plot fluorescence
fig = plt.figure(figsize=(20,3))
ax = fig.add_subplot(1,1,1)
plt.plot(fluo, 'g')
plt.show()

In [ ]:
# Save the results
df = pd.DataFrame(fluo)
df.to_csv(FILE_FLUO, index=False)

In [ ]:
# Correct for camera moving (jumps in fluo caused by camera recentering and capturing more of animal)

# Identify big gaps between consecutive frames
x = len(fluo)-1
ax.axvline(x, color='r')
fig

In [ ]:
fluo = pd.read_csv(FILE_FLUO)
diff = fluo.iloc[x] - fluo.iloc[x-1]
unadjusted = fluo.iloc[:x]
adjusted = fluo.iloc[x:]-diff
fluo_adjusted = pd.concat((unadjusted,adjusted))

plt.figure(figsize=(20,3))
plt.plot(fluo_adjusted, 'g')
plt.show()

In [ ]:
# Save the results again
df = pd.DataFrame(fluo_adjusted)
df.to_csv(FILE_FLUO, index=False)

## Compare midline length and fluorescence

In [ ]:
midlens = pd.read_csv(FILE_LENGTH)

fluo = pd.read_csv(FILE_FLUO)

In [ ]:
# using this to work around nan values in midlens data
def norm_to_zero_one_df(df):
    return (df - df.min()) * 1.0 / (df.max() - df.min())

def norm_to_zero_one_np(arr):
    return (arr - np.min(arr)) * 1.0 / (np.max(arr) - np.min(arr))

In [ ]:
midlens_norm = norm_to_zero_one_df(midlens)
fluo_norm = norm_to_zero_one_np(fluo)

In [ ]:
# Plot overlayed length and fluorescence

timex = np.arange(0, len(midlens)/FPS, 1/FPS)

fig = plt.figure(figsize=(20,3))
ax = fig.add_subplot(1,1,1)
ax_ = ax.twinx()
lns1 = ax.plot(timex, midlens_norm, 'b', label='length')
lns2 = ax.plot(timex, fluo_norm, 'g', label='fluo')
lns = lns1 + lns2
labs = [l.get_label() for l in lns]
ax.legend(lns, labs, loc='best')
ax.set(xlabel='Time [sec]', ylabel='Length/ fluo (normalized)', title=VIDEO_NAME)

# save plot (without highlighting)
plt.savefig(SAVE_DIR + VIDEO_NAME + "_plot.jpg")

In [ ]:
# highlight portion of graph to display alongside movie clip
ax.axvspan(100, 250, color='yellow', alpha=0.5)
# ax.axvspan(500, 550, color='pink', alpha=0.5)
fig